In [1]:
import urllib.request, json
import pandas as pd
import argparse
import numpy as np
from influxdb import InfluxDBClient

In [2]:
from IPython.display import display, HTML
df1=pd.read_csv("C:/Users/Swarupa/Desktop/Terrorism.csv")

C:\Users\Swarupa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4,6,31,33,53,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
del df1["eventid"]

In [4]:
del df1["approxdate"]

In [5]:
del df1["imonth"]
del df1["iday"]

In [7]:
grouped = pd.DataFrame(df1.groupby("iyear").agg(lambda x : x.sum() if x.dtype=='float64' else ' '.join(x))).reset_index()
print(grouped)

    iyear                                        country_txt  \
0    1970  Dominican Republic Mexico Philippines Greece J...   
1    1971  United States West Germany (FRG) United States...   
2    1972  Panama Philippines Kuwait Spain Argentina Unit...   
3    1973  Colombia Ireland Dominican Republic United Kin...   
4    1974  Italy France Italy Bolivia United Kingdom Arge...   
5    1975  Pakistan Spain Spain Spain United Kingdom Neth...   
6    1976  Turkey Turkey Portugal West Germany (FRG) Unit...   
7    1977  Japan Japan Japan France France Spain Italy Sp...   
8    1978  Brazil El Salvador Italy Italy Italy Japan Net...   
9    1979  Japan Colombia Philippines Nicaragua Nicaragua...   
10   1980  West Bank and Gaza Strip United Kingdom United...   
11   1981  Ireland El Salvador Iraq Peru France Somalia S...   
12   1982  Canada Guatemala Iran Colombia Colombia Colomb...   
13   1983  South Africa United Kingdom Netherlands Peru P...   
14   1984  Peru El Salvador El Salvador 

In [8]:
grouped.to_csv("TerrorismByYear.csv",index=False)

In [10]:
def read_data():
    with open('C:/Users/Swarupa/Desktop/SocialWeather/SocialWeather/TerrorismByYear.csv') as f: #Input the name of your csv file here
        return [x.split(',') for x in f.readlines()[1:]]
def read_header():
    with open('C:/Users/Swarupa/Desktop/SocialWeather/SocialWeather/TerrorismByYear.csv') as f: #Input the name of your csv file here
        return [x.split(',') for x in f.readlines()[:1]]

def main(host='localhost', port=8086):       #input your port number which has been set for influx
    """Instantiate the connection to the InfluxDB client."""
    user = 'Swarupa'
    password = ''
    dbname = 'SocialWeather'
    # Temporarily avoid line protocol time conversion issues #412, #426, #431.
    protocol = 'json'

    client = InfluxDBClient(host, port, user, password, dbname)

    print("Create DataFrame");
    b=read_header();
    length=len(b);
    print(b);
    narr=(b[-1])
    narr[-1]=narr[-1].split('\n',1)[0]
    print(narr); # getting /n (new line?,why?)
    print("Create database: " + dbname)
    #client.create_database(dbname)
    a = read_data();
    for metric in a:
        for i in range(len(narr)):
            influx_metric = [{
                'measurement': 'TerrorismByYear',
                'tags': {
                    'items': narr[i]                },
                'time':str(metric[0])+"-12-31"+"T00:00:00Z",
                'fields': {
                    'value': metric[i].split('\n',1)[0]

                }
            }]
            client.write_points(influx_metric)

    
if __name__ == '__main__':
    main()


Create DataFrame
[['iyear', 'country_txt', 'region_txt', 'latitude', 'longitude', 'specificity', 'alternative', 'attacktype1_txt', 'attacktype2', 'attacktype3', 'targtype1_txt', 'targsubtype1', 'natlty1', 'targtype2', 'targsubtype2', 'natlty2', 'targtype3', 'targsubtype3', 'natlty3', 'gname', 'guncertain1', 'guncertain2', 'guncertain3', 'nperps', 'nperpcap', 'claimed', 'claimmode', 'claim2', 'claimmode2', 'claim3', 'claimmode3', 'compclaim', 'weaptype1_txt', 'weapsubtype1', 'weaptype2', 'weapsubtype2', 'weaptype3', 'weapsubtype3', 'weaptype4', 'weapsubtype4', 'nkill', 'nkillus', 'nkillter', 'nwound', 'nwoundus', 'nwoundte', 'propextent', 'propvalue', 'ishostkid', 'nhostkid', 'nhostkidus', 'nhours', 'ndays', 'ransom', 'ransomamt', 'ransomamtus', 'ransompaid', 'ransompaidus', 'hostkidoutcome', 'nreleased', 'dbsource\n']]
['iyear', 'country_txt', 'region_txt', 'latitude', 'longitude', 'specificity', 'alternative', 'attacktype1_txt', 'attacktype2', 'attacktype3', 'targtype1_txt', 'targsubt

KeyboardInterrupt: 